In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install --quiet "pymupdf>=1.18.19"
!pip install --quiet "pdf2image"
!pip install --quiet "pytesseract"
!pip install --quiet "pillow"
!pip install --quiet "opencv-python"
!pip install --quiet "pandas"

ERROR: Could not find a version that satisfies the requirement pymupdf>=1.18.19 (from versions: none)
ERROR: No matching distribution found for pymupdf>=1.18.19


In [3]:
!pip install --quiet "pymupdf==1.22.0"

ERROR: Could not find a version that satisfies the requirement pymupdf==1.22.0 (from versions: none)
ERROR: No matching distribution found for pymupdf==1.22.0


In [5]:
!apt-get -y install poppler-utils  # Required by pdf2image
!pip install --quiet pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 84 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 700 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.8
Ign:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.8
Ign:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.8
Ign:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
Ign:1 http://archive.ubuntu.com/ubuntu jammy-update

In [6]:
# =============================
# STEP 2: Imports
# =============================
import os
import glob
import torch
import pandas as pd
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification

2025-06-15 18:12:32.638954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750011152.874496     312 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750011152.943374     312 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
# =============================
# STEP 3: InvoiceOCRParser Class
# =============================
class InvoiceOCRParser:
    def __init__(self,
                 model_dir: str,
                 processor_dir: str,
                 label_map: dict = None,
                 device: str = None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.processor = LayoutLMv3Processor.from_pretrained(processor_dir, local_files_only=True)
        self.model = LayoutLMv3ForTokenClassification.from_pretrained(model_dir, local_files_only=True)
        self.model.to(self.device)
        self.model.eval()

        self.label_list = label_map or ['O', 'B-INVOICE_ID', 'I-INVOICE_ID',
                                        'B-VENDOR', 'I-VENDOR',
                                        'B-DATE', 'I-DATE',
                                        'B-LINE_ITEM', 'I-LINE_ITEM',
                                        'B-TAX', 'I-TAX',
                                        'B-TOTAL', 'I-TOTAL']

    def _pdf_to_images(self, pdf_path):
        return convert_from_path(pdf_path, dpi=300)

    def _load_images_from_folder(self, folder_path):
        file_list = glob.glob(os.path.join(folder_path, "*"))
        result = []
        for file_path in file_list:
            ext = file_path.lower().split('.')[-1]
            if ext in ['jpg', 'jpeg', 'png']:
                pil_img = Image.open(file_path).convert("RGB")
                result.append((os.path.basename(file_path), [pil_img]))
            elif ext == 'pdf':
                try:
                    imgs = self._pdf_to_images(file_path)
                    result.append((os.path.basename(file_path), imgs))
                except Exception as e:
                    print(f"Could not parse PDF {file_path}: {e}")
        return result

    def _image_to_words_boxes(self, image):
        ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        words, boxes = [], []
        for i, word in enumerate(ocr_data['text']):
            if word.strip() == "" or int(ocr_data['conf'][i]) < 0:
                continue
            x, y, w, h = ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i]
            words.append(word)
            boxes.append([x, y, x + w, y + h])
        return words, boxes

    def _normalize_boxes(self, boxes, width, height):
        norm = []
        for box in boxes:
            x0, y0, x1, y1 = box
            norm.append([
                int(1000 * x0 / width),
                int(1000 * y0 / height),
                int(1000 * x1 / width),
                int(1000 * y1 / height)
            ])
        return norm

    def _predict_tokens(self, words, boxes, image):
        encoding = self.processor(
            image, words, boxes=boxes, return_tensors="pt", truncation=True,
            max_length=512, padding="max_length", return_attention_mask=True
        )
        encoding = {k: v.to(self.device) for k, v in encoding.items()}
        with torch.no_grad():
            outputs = self.model(**encoding).logits
            probs = outputs.softmax(dim=-1)
            preds = torch.argmax(probs, dim=-1).squeeze().tolist()
        tokens = self.processor.tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze())
        labels = [self.label_list[p] if p < len(self.label_list) else 'O' for p in preds]
        word_ids = encoding.get('word_ids') or self.processor.tokenizer(
            words, boxes=boxes, add_special_tokens=True, padding='max_length',
            truncation=True, max_length=512
        ).word_ids()
        out = []
        last_word = None
        for i, (label, tok, word_idx) in enumerate(zip(labels, tokens, word_ids)):
            if word_idx is not None and word_idx != last_word:
                last_word = word_idx
                out.append((label, words[word_idx], boxes[word_idx]))
        return out

    def _extract_fields(self, label_word_box_tuples):
        data = {
            'InvoiceID': '',
            'Vendor': '',
            'Date': '',
            'LineItems': [],
            'Tax': '',
            'Total': ''
        }
        current_field = None
        field_buffer = {k: [] for k in ['INVOICE_ID', 'VENDOR', 'DATE', 'LINE_ITEM', 'TAX', 'TOTAL']}
        for label, word, _ in label_word_box_tuples:
            if label.startswith('B-'):
                current_field = label[2:]
                field_buffer[current_field].append(word)
            elif label.startswith('I-') and current_field:
                field_buffer[current_field].append(word)
            else:
                current_field = None
        data['InvoiceID'] = ' '.join(field_buffer['INVOICE_ID']).strip()
        data['Vendor'] = ' '.join(field_buffer['VENDOR']).strip()
        data['Date'] = ' '.join(field_buffer['DATE']).strip()
        data['Tax'] = ' '.join(field_buffer['TAX']).strip()
        data['Total'] = ' '.join(field_buffer['TOTAL']).strip()
        raw_lis = ' '.join(field_buffer['LINE_ITEM']).strip()
        data['LineItems'] = [raw_lis] if raw_lis else []
        return data

    def process_folder(self, folder_path, display_log=True):
        image_list = self._load_images_from_folder(folder_path)
        results = []
        for filename, images in image_list:
            for page_no, img in enumerate(images):
                words, boxes = self._image_to_words_boxes(img)
                if not words:
                    if display_log:
                        print(f"Could not OCR {filename} page {page_no + 1}, skipping.")
                    continue
                norm_boxes = self._normalize_boxes(boxes, img.width, img.height)
                label_word_box_tuples = self._predict_tokens(words, norm_boxes, img)
                parsed = self._extract_fields(label_word_box_tuples)
                result_row = dict(File=filename, Page=page_no + 1)
                result_row.update(parsed)
                results.append(result_row)
                if display_log:
                    print(f"Parsed {filename} (page {page_no + 1}): {parsed}")
        return pd.DataFrame(results)


In [15]:
!apt-get -y install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 84 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 700 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.8
Ign:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.8
Ign:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.8
Ign:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
Ign:1 http://archive.ubuntu.com/ubuntu jammy-update

In [24]:
!pip install -q transformers

In [29]:
from transformers import AutoModel, AutoProcessor

# Download model and processor from Hugging Face
model = AutoModel.from_pretrained("microsoft/layoutlmv3-base")
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base")

# Save them locally
model.save_pretrained("layoutlmv3-base")
processor.save_pretrained("layoutlmv3-base")

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

[]

In [32]:
!zip -r layoutlmv3-base.zip layoutlmv3-base

  adding: layoutlmv3-base/ (stored 0%)
  adding: layoutlmv3-base/preprocessor_config.json (deflated 50%)
  adding: layoutlmv3-base/config.json (deflated 56%)
  adding: layoutlmv3-base/merges.txt (deflated 53%)
  adding: layoutlmv3-base/tokenizer_config.json (deflated 75%)
  adding: layoutlmv3-base/tokenizer.json (deflated 82%)
  adding: layoutlmv3-base/special_tokens_map.json (deflated 85%)
  adding: layoutlmv3-base/vocab.json (deflated 59%)
  adding: layoutlmv3-base/model.safetensors (deflated 7%)


In [33]:
import os
os.listdir()

['layoutlmv3-base', '.virtual_documents', 'layoutlmv3-base.zip']

In [35]:
# Replace this with your actual parser class implementation if needed
parser = InvoiceOCRParser(
    model_dir='/kaggle/working/layoutlmv3-base',
    processor_dir='/kaggle/working/layoutlmv3-base',
)

# Run inference on the folder of images
invoice_df = parser.process_folder('/kaggle/input/invoice_images/')  # Replace with your dataset
display(invoice_df)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at /kaggle/working/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


""
